In [ ]:
from qiskit import QuantumCircuit, transpile, execute
from qiskit_aer import AerSimulator
import numpy as np
from typing import List, Tuple



1) Create Receiver class

In [ ]:
#Bit flip noise version
class Receiver:
    def __init__(self, n_bits, noise_prob=0.05):
        self.receiver_bases = np.random.randint(2, size=n_bits)
        self.noise_prob = noise_prob

    def measure_qubits_with_noise(self, circuits):
        backend = AerSimulator()
        measured_bits = []

        for i, qc in enumerate(circuits):
            noisy_qc = qc.copy()

            # Apply bit-flip noise
            if np.random.rand() < self.noise_prob:
                noisy_qc.x(0)

            # Bob's basis change
            if self.receiver_bases[i] == 1:
                noisy_qc.h(0)

            # Measurement
            noisy_qc.measure(0, 0)

            # Run on AerSimulator
            transpiled_qc = transpile(noisy_qc, backend)
            job = backend.run(transpiled_qc, shots=1, memory=True)
            result = job.result()
            bit = int(result.get_memory()[0])
            measured_bits.append(bit)

        return measured_bits

In [ ]:
#Depolarizing noise version

import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

class Receiver:
    def __init__(self, n_bits, depol_prob=0.05):
        self.receiver_bases = np.random.randint(2, size=n_bits)
        self.depol_prob = depol_prob

    def measure_qubits_with_depolarizing_noise(self, circuits):
        backend = AerSimulator()
        measured_bits = []

        for i, qc in enumerate(circuits):
            noisy_qc = qc.copy()

            # --- Depolarizing noise ---
            if np.random.rand() < self.depol_prob:
                noise_gate = np.random.choice(['X', 'Y', 'Z'])
                if noise_gate == 'X':
                    noisy_qc.x(0)
                elif noise_gate == 'Y':
                    noisy_qc.y(0)
                else:
                    noisy_qc.z(0)

            # Apply Bob's basis
            if self.receiver_bases[i] == 1:
                noisy_qc.h(0)

            # Measurement
            noisy_qc.measure(0, 0)
            transpiled_qc = transpile(noisy_qc, backend)
            job = backend.run(transpiled_qc, shots=1, memory=True)
            result = job.result()
            bit = int(result.get_memory()[0])
            measured_bits.append(bit)

        return measured_bits


In [ ]:
#random generator 
#measure_function which will measure the sender qubit
#Compare bases and sift the key

In [ ]:
#Ideal version with no noise
class Receiver:
    def __init__(self, n_bits):
        self.receiver_bases = np.random.randint(2, size=n_bits)
        self.measurements = []
    
    def measure_qubits(self, circuits):  # Added self parameter and fixed indentation
        simulator = AerSimulator()
        bob_results = []

        for i, qc in enumerate(circuits):
             from qiskit import Aer, execute
        backend = Aer.get_backend('qasm_simulator')
        for qc, basis in zip(circuits, self.receiver_bases):
            if basis == 1:
                qc.h(0)  # Convert X-basis to Z-basis for measurement
            qc.measure(0, 0)
            result = execute(qc, backend, shots=1).result()
            counts = result.get_counts()
            self.measurements.append(int(list(counts.keys())[0]))
        return self.measurements